In [2]:
# installing all dependencies required for the notebook
%pip install -r requirements.txt

  Using cached blis-0.4.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl (4.0 MB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached certifi-2020.11.8-py2.py3-none-any.whl (155 kB)
  Using cached cymem-2.0.4-cp37-cp37m-macosx_10_9_x86_64.whl (31 kB)
ERROR: Could not find a version that satisfies the requirement en-core-web-sm==2.3.1 (from -r requirements.txt (line 9)) (from versions: none)
ERROR: No matching distribution found for en-core-web-sm==2.3.1 (from -r requirements.txt (line 9))
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pyspellchecker
from spellchecker import SpellChecker

Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd

#import data
data = pd.read_csv("data/train_preprocessed.csv")

,id,keyword,location_data,text_data,target_data,drinking,courtney,@blazerfan,#speakingfromexperience,@cortezera,...,http://t.co/ixw2cutk1c,#oklahoma,http://t.co/s4sicmyrmh,airplane,95,aaarrrgghhh,https://t.co/cvkqigr1az,screams~,#silver,http://t.co/cubc0nq6fd
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
## initialize spellchecker and test on a few words
spell = SpellChecker(distance=1)

test = ['na', 'lief']
for word in test:
    # Get the one `most likely` answer
    print(spell.correction(word))

na
life


In [40]:
#store all of the columns that encode words (all but the first 4, which contain the original tweet data)
bag_of_words = data.columns[4:]

In [41]:
## for each word (one-hot encoded as column names) replace with spellcheck version
column_map = {} #dictionary to store map for corrected words
n_corrected = 0 #count number of corrections made

#iterate through all columns that represent tweet words 
for word in bag_of_words:
    #filter out hashtags, mentions, and urls
    if '#' not in word and 'http://' not in word and '@' not in word:
        #use spell check to get correct version of word
        corrected = spell.correction(word)
        #if word is corrected by spellchecker:
        if word != corrected:
            #store correct version in dictionary, then print and count
            column_map[word] = corrected
            n_corrected += 1
            print(word, 'corrected to ', corrected)

rrected to  bout
firefigther corrected to  firefighter
mgs2 corrected to  mgs
|lauren corrected to  lauren
dlh corrected to  doh
rockin corrected to  rocking
bardo corrected to  bard
minaj corrected to  mina
x'mas corrected to  xmas
knowlddge corrected to  knowledge
sanitize corrected to  sanitized
forsee corrected to  foresee
faq corrected to  far
-honey corrected to  honey
5so corrected to  so

  corrected to  y 
radneck corrected to  redneck
pajama corrected to  panama
wii corrected to  win
childress corrected to  childless
-loses corrected to  loses
racco corrected to  zacco
lith corrected to  with
ps4 corrected to  ps
-anonymous corrected to  anonymous
wce corrected to  we
osama corrected to  osaka
myelf corrected to  myself
postere corrected to  posters
rousey corrected to  roused
r3do corrected to  rodo
superintende corrected to  superintend
arreat corrected to  arrest
yoenis corrected to  younis
indah corrected to  ndah
'the corrected to  the
lbr corrected to  br
optione correc

In [42]:
#print out percentage of words corrected
print('% corrected: ', n_corrected / len(bag_of_words))

% corrected:  0.07122426970506869


In [43]:
#rename columns to corrected version
data.rename(columns=column_map, inplace=True)

In [44]:
print(len(data.columns), len(set(data.columns))) #after combining these numbers should be the same

21331 20554


In [51]:
## combine corrected words that are now duplicates
columns_copy = list(data.columns) #copy column list so that it isn't affected by deleting duplicateees
for i in range(len(columns_copy)):
    this_word = columns_copy[i]
    #check if word is duplicate by seeing if it appears in the list again
    if this_word in columns_copy[i+1:]:
        #combine duplicate columns by taking the maximum value; 
        #if any row contains a 1 for a spelling variation, the combined output will also contain a 1
        combined_columns = data[this_word].max(axis=1, skipna=False)
        #delete duplicate columns
        data.drop(labels=this_word, axis=1, inplace=True)
        #insert combined column into dataset
        data[this_word] = combined_columns
        print(this_word, 'duplicate')
        print(len(columns_copy), len(data.columns))

run duplicate
20576 20575
or duplicate
20576 20574
crisis duplicate
20576 20573
currency duplicate
20576 20572
breath duplicate
20576 20571
learn duplicate
20576 20570
big duplicate
20576 20569
tax duplicate
20576 20568
humanity duplicate
20576 20567
sunday duplicate
20576 20566
loud duplicate
20576 20565
ask duplicate
20576 20564
paris duplicate
20576 20563
nbc duplicate
20576 20562
relive duplicate
20576 20561
combat duplicate
20576 20560
spark duplicate
20576 20559
then duplicate
20576 20558
fleet duplicate
20576 20557
future duplicate
20576 20556
wife duplicate
20576 20555
wrong duplicate
20576 20554


In [52]:
print(len(data.columns), len(set(data.columns))) #after combining these numbers should be the same

20554 20554


In [54]:
##write spellchecked to csv
data.to_csv('data/spellchecked_train.csv')